# What's happening here?

4_link_video_behavior.ipynb

Run after 1_, 2_, and 3_

Made in environment **PMS_Study**

## TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math
import time
import json
import statistics

import features as ft

from tqdm import tqdm
from pathlib import Path

import re

In [2]:
# Create all relevant directories
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)

# Load base data
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")
allData = pd.read_csv(data_all_dir + "allPMSdata.csv", encoding = "ISO-8859-1")
timings = pd.read_csv(data_all_dir + "timings.csv", encoding = "ISO-8859-1")
timings = timings.rename(columns={"ï»¿ImageGroup": "ImageGroup"}, index={'ONE': 'Row_1'}) # Rename first column, because was formatted weird

redoOpenface = 1 # Set to 1 if you want to re-do OpenFace processing. For instance if you added features, or changed methods

In [3]:
timings

,ImageGroup,Date,pptnumber,StartTime,EndTime,OtherComments,Unnamed: 6,Exclusion,TestMoment,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,B,8/27/2020,14,26,0,"looks to the side, tv playing",NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,B,9/28/2020,66,16,338,looks at phone: Same ppt as 14,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN
2,B,8/31/2020,106,36,0,NaN,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN
3,B,9/23/2020,119,74,407,drinking tea,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN
4,B,07/09/2021,123,31,314,typing (texting0),NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,A,8/31/2021,674,104,441,"in background, there's a picture of a face (no...",NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN
521,A,07/08/2021,676,128,425,hair bit over left eye,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
522,A,9/24/2021,678,0,0,no beeps,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN
523,A,9/30/2021,679,12,311,"eyebrow piercing, chin not visible",NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Get all OpenFace files
os.chdir(data_dir)
AFiles = glob.glob("10001/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BFiles = glob.glob("10002/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory

# Opening JSON file with all video duration information
f = open(data_all_dir + 'video_durations.json',)
durationsdata = json.load(f)
f.close()

In [6]:
def get_openfacestuff(fileDir):
    print(fileDir)
    if len(indices) > 1:
        print("Too many OpenFace files for ppt: " + str(pptnum) + " at moment " + moment)
    else: # If we get to here - the party really starts
        
        if math.isnan(allData['AU12'][idx[0]]) or redoOpenface == 1: # Check if OpenFace values have already been added, of whether manually set at top of script to re do this (so it doesnt happen unneccessarily
            openface = pd.read_csv(fileDir, encoding = "ISO-8859-1") # Read openface output csv

            # Get corresponding video file duration 
            splits = fileDir.split('/video_features\\')
            fileduration = [value for key, value in durationsdata.items() if splits[0] in key.lower() and splits[1][23:len(splits[1])-4] in key.lower()] # Very weird formatting, but when checking for directory (10001/10002) and only the date (something with curly bracktes messes it up. 
            frameduration = fileduration[0] / len(openface)
            openface.insert (3, "timestamp2", np.arange(0, fileduration[0], frameduration).tolist()[0:len(openface)]) # Add new timestamps right after the first timestamp || We're taking [0:len(openface)] here because due to rounding error, the newly created timestamps might be one longer than th array. 
            videoOffset
            behaviorOffset = allData['startBeep'][idx[0]]
            correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
            videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

            correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
            videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()

            # Loop over all the trials and get the appropriate values - add to allData
            for trial in range(0,len(videoStarts)):
                vidStartIdx = openface.iloc[(openface['timestamp2']-videoStarts[trial]).abs().argsort()[:1]].index.tolist()[0] # Get start idx in openface
                vidEndIdx = openface.iloc[(openface['timestamp2']-videoEnds[trial]).abs().argsort()[:1]].index.tolist()[0] # Get end idx in openface

                # Create mean value between these idx's and add to big dataframe
                allData.loc[idx[trial], 'AU12'] = openface[' AU12_r'][vidStartIdx:vidEndIdx].mean() 
                allData.loc[idx[trial], 'AU15'] = openface[' AU15_r'][vidStartIdx:vidEndIdx].mean() 
                
                allData.loc[idx[trial], 'arousal'] = compute_arousal(openface[vidStartIdx:vidEndIdx])['mean_Arousal']
                allData.loc[idx[trial], 'blinkrate'] = compute_blink_rate(openface[vidStartIdx:vidEndIdx])
                allData.loc[idx[trial], 'PD_Mean'] = compute_PD_features(openface[vidStartIdx:vidEndIdx])['mean_PD']
                allData.loc[idx[trial], 'PD_Std'] = compute_PD_features(openface[vidStartIdx:vidEndIdx])['std_PD']
                allData.loc[idx[trial], 'PD_Max'] = compute_PD_features(openface[vidStartIdx:vidEndIdx])['max_PD']
                
                allData.loc[idx[trial], 'PoseMeasures'] = compute_head_motion(openface[vidStartIdx:vidEndIdx])

                allData.loc[idx[trial], 'happy'] = compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Happy']
                allData.loc[idx[trial], 'sad'] = compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Sad']
                allData.loc[idx[trial], 'disgust'] = compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Disgust']
                allData.loc[idx[trial], 'anger'] = compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Angry']
                allData.loc[idx[trial], 'scared'] = compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Scared']

In [5]:
# The BIG loop where all the magic happens

index = 0
for pptnum in tqdm(timings["pptnumber"]): # Loop over all ppt's that had a video available (therefore present in timings.csv)
    if '514' not in str(pptnum) and '583' not in str(pptnum) and '342' not in str(pptnum) and '4888' not in str(pptnum): # '&': This is something that gives errors, '383': Corrupted file, '467': same, '495': same, '502': same

        print("processing pptnum: " + str(pptnum))
        idx = np.where(allData['ID'] == int(pptnum))[0]
        if timings["Exclusion"][index] == 1: # Only continue the loop if there is no Exclusion marked
            print("Exclusion file for ppt: " + str(pptnum) + " at moment: " + timings["ImageGroup"][index])
        else:
            videoOffset = float(timings["StartTime"][index])
            if videoOffset == 0: # if a timing start point was not present, it was set to 0. We don't process those because we cant sync them
                print('start time for this ppt is set to zero')
            else:
                if len(idx) < 1: # Check if occurences in datafile exist for this participant/video file
                    print("No input data found for video file of pptnum: " + str(pptnum) + " with video file " + timings["ImageGroup"][index] )
                else:           
                    moment = timings["ImageGroup"][index]
                    if moment ==  'A':
                        indices = [i for i, s in enumerate(AFiles) if str(pptnum) in s]
                        try:
                            fileDir = AFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)
                    elif moment == 'B':
                        indices = [i for i, s in enumerate(BFiles) if str(pptnum) in s]
                        try:
                            fileDir = BFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)

                    else:
                        print("Some error with testing moment")
                        break

    index += 1 # Create counter
    if index > 10:
        break

  2%|█▎                                                                              | 9/525 [00:00<00:00, 1007.12it/s]


processing pptnum: 14
Exclusion file for ppt: 14 at moment: B
processing pptnum: 66
Exclusion file for ppt: 66 at moment: B
processing pptnum: 106
No input data found for video file of pptnum: 106 with video file B
processing pptnum: 119
No input data found for video file of pptnum: 119 with video file B
processing pptnum: 123
No input data found for video file of pptnum: 123 with video file B
processing pptnum: 124
No input data found for video file of pptnum: 124 with video file B
processing pptnum: 125
No input data found for video file of pptnum: 125 with video file B
processing pptnum: 126
No input data found for video file of pptnum: 126 with video file B
processing pptnum: 127
start time for this ppt is set to zero
processing pptnum: 129
10002/video_features\video_{ParticipantNo}_129_9-10-2020.csv


NameError: name 'allData' is not defined

In [5]:
ft.get_openfacestuff()

TypeError: get_openfacestuff() missing 3 required positional arguments: 'fileDir', 'indices', and 'test'

In [122]:
videoOffset = float(timings['StartTime'][0])
videoOffset

correctedStarts
# ((correctedStarts + videoOffset) / 1000).tolist()

3718       4726
3719      20895
3720      34010
3721      47683
3722      60814
3723      73294
3724      85816
3725      98923
3726     111116
3727     124704
3728     137785
3729     151708
3730     163725
3731     176150
3732     189041
3733     200107
3734     213556
3735     226230
3736     238952
3737     251938
3738     263958
3739     276192
11792    -73574
11793    -60274
11794    -45511
11795    -28566
11796    -14695
11797     -1709
11798     13117
11799     26678
11800     41464
11801     55070
11802     67491
11803     78492
11804     91073
11805    103613
11806    123692
11807    135209
11808    147157
11809    159894
11810    171167
11811    182742
11812    194368
11813    210096
Name: startView, dtype: int64

In [126]:
print(videoOffset)
((correctedStarts + videoOffset) / 1000).tolist()

0.26


[4.72626,
 20.895259999999997,
 34.01026,
 47.683260000000004,
 60.814260000000004,
 73.29426,
 85.81626,
 98.92326,
 111.11626,
 124.70425999999999,
 137.78526000000002,
 151.70826,
 163.72526000000002,
 176.15026,
 189.04126000000002,
 200.10726,
 213.55626,
 226.23026000000002,
 238.95226,
 251.93826,
 263.95826,
 276.19226000000003,
 -73.57374,
 -60.27374,
 -45.51074,
 -28.56574,
 -14.69474,
 -1.70874,
 13.11726,
 26.678259999999998,
 41.46426,
 55.070260000000005,
 67.49126,
 78.49226,
 91.07325999999999,
 103.61326,
 123.69225999999999,
 135.20926,
 147.15726,
 159.89426,
 171.16726,
 182.74226000000002,
 194.36826000000002,
 210.09626]

In [79]:
os.chdir(os.path.join(data_dir,date_dir))
allData.to_csv('allPMSdata.csv')

In [82]:
allData

,Unnamed: 0,ID,Age,FirstMenstrual,MenstrualDuration,Exclusie,TrueFollicular,TrueLuteal,Isala,Order,...,stopView,stopResponse,startBeep,endBeep,AU12,AU15,happy,sad,disgust,anger
0,0,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,113826,118663,102071,430411,0.264500,0.253167,0.132500,0.222333,0.126833,0.227708
1,1,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,127671,131315,102071,430411,0.010331,0.256961,0.005166,0.169319,0.160442,0.041202
2,2,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,140322,145072,102071,430411,0.098066,0.340829,0.049033,0.240350,0.182265,0.057265
3,3,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,154078,160026,102071,430411,0.022265,0.475414,0.011133,0.327882,0.257762,0.043260
4,4,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,169032,176107,102071,430411,0.013611,0.157778,0.006806,0.125630,0.085333,0.036472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,15857,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,342239,346285,83286,400540,0.000000,0.007207,0.000000,0.293706,0.003603,0.318450
15858,15858,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,355296,360904,83286,400540,0.000000,0.005167,0.000000,0.301833,0.002667,0.340250
15859,15859,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,369909,373091,83286,400540,0.000000,0.053855,0.000000,0.340987,0.026927,0.331927
15860,15860,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,382101,387019,83286,400540,0.000000,0.050838,0.000000,0.338417,0.025419,0.342249
